# Clustering et word2vec

Sur la base des éléments méthodologiques et des enseignements techniques présentés lors du cours théorique, il est demandé dans le cadre de ce TP :
- d’effectuer un clustering des bulletins pour une décennie au choix et d’interpréter les résultats
- d’entraîner un modèle word2vec sur l’ensemble des bulletins et d’explorer les relations entre vecteurs

Pour ce faire, vous utiliserez différentes librairies Python vues au cours comme scikit-learn et gensim.

## 1. Clustering

## 2. Word2Vec

#### Librairies nécessaires

In [1]:
import sys
import os

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

In [2]:
sw = stopwords.words("french")

In [3]:
with open(infile, encoding="utf-8", errors="backslashreplace") as f:
    print(type(f))

NameError: name 'infile' is not defined

In [ ]:
kept_lines = []

with open(outfile, 'w', encoding="utf-8") as output:
    with open(infile, encoding="utf-8", errors="backslashreplace") as f:
        type(f)
        content = f.readlines()
        n_lines = len(content)
        for i, line in enumerate(content):
            for w in line:
                if len(w) > 2 and w.isalph() and w.lower() not in sw:
                    kept_lines.append(" ".join(w))
        output.write(kept_lines + "\n")
print("Done")

In [ ]:
kept = []
lines = []
def clean_text(my_file):
    output_path = f"../data/sents_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(my_file, encoding='utf-8') as f:
        for line in f.read():
            for w in line:
                if len(w) > 1 and w.isalpha() and w.lower() not in sw:
                    kept.append(w)
            kept_lines = " ".join(kept)
        lines.append(kept_lines)
        output.write(lines)
    return f'Output has been written in {output_path}!'

In [ ]:
def clean_text(my_file):
    output_path = f"../data/sents_2_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(my_file, encoding='utf-8') as f:
        text = f.read()
        kword = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in kword if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string + "\n")
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text(infile)

In [ ]:
cleaned_infile = f"../data/sents_2_clean.txt"

In [ ]:
text_selected = open(os.path.join(f'../data/sents_2_clean.txt'), 'r').read()
print(len(text_selected))
print(text_selected[:100])

#### Chargement des phrases

In [4]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

#### Fichier à nettoyer

In [5]:
infile = f"../data/sents_2.txt"

In [6]:
sentences = MySentences(infile)

#### Création des bigrammes

In [7]:
bigram_phrases = Phrases(sentences)

In [8]:
len(bigram_phrases.vocab.keys())

5978799

In [9]:
# exploration des bigrammes

key_ = list(bigram_phrases.vocab.keys())[145]
print(key_)

maladie


In [10]:
# score de la clé

bigram_phrases.vocab[key_]

2929

In [11]:
print(list(bigram_phrases.vocab.keys())[:10])

['v', 'i', 'v_i', 'l', 'i_l', 'l_l', 'e', 'l_e', 'de', 'e_de']


In [12]:
%time bigram_phrases[sentences]

CPU times: user 6.83 ms, sys: 0 ns, total: 6.83 ms
Wall time: 5.87 ms


In [13]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

In [14]:
%time bigram_phraser[sentences]

CPU times: user 2.35 ms, sys: 0 ns, total: 2.35 ms
Wall time: 2.73 ms


In [15]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [16]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

In [17]:
quadrigram_phrases = Phrases(trigram_phraser[sentences])

In [18]:
quadrigram_phraser = Phraser(phrases_model=quadrigram_phrases)

In [19]:
pentagram_phrases = Phrases(quadrigram_phraser[sentences])

In [20]:
pentagram_phraser = Phraser(phrases_model=pentagram_phrases)

In [21]:
corpus = list(pentagram_phraser[quadrigram_phraser[trigram_phraser[bigram_phraser[sentences]]]])

In [22]:
print(corpus[:10])

[['v', 'i', 'l', 'l', 'e', 'de', 'bruxelles', 'bulletin', 'ires', '8eanas', 'dl', '!'], ['conseil_communal', 'annee', '1847', '.'], ['au', 'ville', 'de', 'b', 'r', 'u', 'x', 'e', 'l', 'l', 'e', 's', '.'], ['bulletin', 'conseil', 'aes', 'seances', 'communal', '.'], ['annee', '1847', '.'], ['bruxelles', ',', 'imprimerie', 'd', 'e', 'j', '.'], ['h', '.', 'b', 'r', 'i', 'a', 'r', 'd', ',', 'rite', 'n', 'e', 'u', 'v', 'e', ',', '3', '1', ',', 'faubourg', 'de', 'n', 'a', 'm', 'u', 'r', ',', '1', '84', '8', 'de', '!'], ['du', 'consei', 'dibi', 'e', '.', '-', 'communication', 'conclusions', 'de', 'la', 'section', 'des', 'du', 'nouvel_hospice', 'pour', 'les', 'av', 'enraisonde', 'l', "'", 'absence', '&', 'maladie', '.', 'le', 'conseil', 'ajourne', 'leurs', 'de', 'pierre', 'el', 'marchai', 'cles', 'des', 'taxes', 'communale', "'", 'bieniaance', 'eldeseianv', 'il', 'est', 'donne', 'communie', ';', 'mandant', 'le', 'o', 'p', 'fa', 'gnant', 'l', "'", 'envoi', 'de', 'leur', 'bn', 'par', 'l', "'", 'e

In [23]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 2h 20min 49s, sys: 9min 36s, total: 2h 30min 25s
Wall time: 44min 14s


In [24]:
outfile = f"../data/bulletins_tp3_1.model"
model.save(outfile)

In [2]:
model = Word2Vec.load("../data/bulletins_tp3_1.model")

#### Exploration du modèle

In [5]:
model.wv.similarity("boucher", "boulanger")

0.57089794

In [30]:
model.wv.similarity("homme", "femme")

0.42860717

In [21]:
model.wv.similarity("chat", "chien")

0.5798148

In [22]:
model.wv.similarity("voiture", "carrosse")

0.28735888

In [25]:
model.wv.similarity("voiture", "chien")

0.42339402

In [6]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.5818007588386536),
 ('nivelles', 0.5182838439941406),
 ('liege', 0.5148740410804749),
 ('bruxeles', 0.4937210977077484),
 ('bruxelle', 0.4917224645614624),
 ('jrvice', 0.471939355134964),
 ('xelles', 0.4709879159927368),
 ('huy', 0.47084033489227295),
 ('prague', 0.46914100646972656),
 ('binche', 0.46828752756118774)]

In [27]:
model.wv.most_similar("platonisme", topn=10)

[('aristotelisme', 0.8893957734107971),
 ('symbolis', 0.8818884491920471),
 ('centum', 0.8812199831008911),
 ('christiano', 0.879470705986023),
 ('147437', 0.8793374300003052),
 ('66637', 0.8787049055099487),
 ('153125', 0.8774873614311218),
 ('gewidmet', 0.8770879507064819),
 ('43451', 0.8768373727798462),
 ('122571', 0.8763118982315063)]

In [7]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.5035298466682434), ('londres', 0.494385689496994), ('milan', 0.48436862230300903), ('tournai', 0.47917115688323975), ('huy', 0.4791364371776581), ('verviers', 0.470472127199173), ('prague', 0.4691561460494995), ('ostende', 0.46747785806655884), ('hambourg', 0.4582105278968811), ('paris', 0.4556889533996582)]


In [18]:
print(model.wv.most_similar(positive=['roi', 'femme'], negative=['homme']))

[('mere', 0.4184131324291229), ('eoi', 0.4026956558227539), ('boi', 0.39582201838493347), ('fille', 0.3846323490142822), ('famille_royale', 0.3765176236629486), ('eclale', 0.37051498889923096), ('zeepaard', 0.3684881925582886), ('sieur_van_eesbeeck', 0.36782413721084595), ('mgr', 0.3621143698692322), ('princesse_marie', 0.35621941089630127)]


In [15]:
print(model.wv.most_similar(positive=['fidelite', 'homme'], negative=['femme']))

[('obeissance', 0.538389265537262), ('<<_je_jure', 0.5057224631309509), ('je_jure_fidelite', 0.4389154016971588), ('inalterableattachement', 0.4325920641422272), ('hommage_particulier', 0.4223836660385132), ('peuple_belge', 0.42071980237960815), ('desinteressement', 0.4178043603897095), ('serment_suivant', 0.41470056772232056), ('serment', 0.4126855134963989), ('juste_hommage', 0.4065362513065338)]


In [14]:
print(model.wv.most_similar(positive=['femme', 'homme'], negative=['fidelite']))

[('individu', 0.5701825618743896), ('enfant', 0.5563074350357056), ('ouvrier', 0.5529573559761047), ('cet_homme', 0.48627912998199463), ('indigent', 0.48061877489089966), ('orphelin', 0.4713592529296875), ('artiste', 0.47036853432655334), ('fille', 0.46726277470588684), ('malade', 0.4668050706386566), ('jeune_fille', 0.465228408575058)]


In [17]:
print(model.wv.most_similar(positive=['urinoir', 'femme'], negative=['homme']))

[('fenetre', 0.5346331000328064), ('mansarde', 0.5246878862380981), ('cloison', 0.520297110080719), ('terrasse', 0.5074345469474792), ('loge', 0.5059982538223267), ('douche', 0.49894893169403076), ('cave', 0.49847882986068726), ('chaudiere', 0.49798569083213806), ('pouponniere', 0.4923699200153351), ('laverie', 0.4922226071357727)]
